## Neural Network

### Notations

- $L$ : 神经网络的层数。
- $n_l$ : 神经网络第 `l` 层的神经元个数。
- $\textbf{W}^{(l)} $: 从第 `l - 1`层到第 `l` 层权重矩阵, $\textbf{W}^{(l)} \in \mathbb{R}^{n_l \times n_{l - 1}}$。
- $w^{(l)}_{ij}$: 从第 `l - 1`层第 `j` 个神经元到 `l` 层第 `i` 个神经元的权重。
- $\textbf{b}^{(l)}$: 从第 `l - 1`层到第 `l` 层的偏置, $\textbf{b}^{(l)} \in \mathbb{R}^{n_l \times 1}$。
- $\textbf{z}^{(l)}$: 第 `l` 层的神经元状态，$\textbf{z}^{(l)} \in \mathbb{R}^{n_l \times 1}$。
- $\textbf{a}^{(l)}$: 第 `l` 层的神经元输出， $\textbf{a}^{(l)} \in \mathbb{R}^{n_l \times 1}$。
- $f(z^{(l)})$: 第 `l` 层的激活函数。

### FP
主要用于预测

### BP

1. 输出层参数更新公式

2. 隐藏层参数更新公式

### 实现

1. 参数初始化
2. 激活函数
3. 激活函数导数
4. evaluate(test_data)
5. FP 
6. model(train_data, epoches, alpha, mini_batch_size, test_data)
7. update_mini_batch()
8. BP


## 神经网络的优化方法
1. 参数初始化
2. 激活函数梯度消失
3. dropout
